In [1]:
%reload_ext autoreload
%autoreload 2
%reload_ext notexbook
%texify

#### Generating Evaluation data using 30 random segmentation masks

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import cellbgnet
import cellbgnet.utils
from cellbgnet.datasets import DataSimulator
from cellbgnet.utils.hardware import cpu, gpu
from cellbgnet.model import CellBGModel
from cellbgnet.simulation.psf_kernel import SMAPSplineCoefficient
from cellbgnet.generic.emitter import EmitterSet
from cellbgnet.train_loss_infer import generate_probmap_cells
from skimage.io import imread
from skimage.measure import label
import random
import edt
from skimage.filters import gaussian
from scipy.ndimage import rotate
import random
import pickle
import pathlib
from pathlib import Path
%matplotlib qt5

In [3]:
param_file = '../../cellbgnet/utils/reference_files/reference.yaml'
param = cellbgnet.utils.param_io.ParamHandling().load_params(param_file)

In [4]:
psf_params = param.PSF.to_dict()
simulation_params = param.Simulation.to_dict()
hardware_params = param.Hardware.to_dict()
train_size = simulation_params['train_size']

In [5]:
with open(param.Simulation.edt_noise_map_path, 'rb') as fp:
    edt_noise_map = pickle.load(fp)

In [6]:
model = cellbgnet.model.CellBGModel(param)

training sliding windows on camera chip:
Area num: 0, field_xy: [0, 127, 0, 127]
Area num: 1, field_xy: [114, 241, 0, 127]
Area num: 2, field_xy: [228, 355, 0, 127]
Area num: 3, field_xy: [342, 469, 0, 127]
Area num: 4, field_xy: [456, 583, 0, 127]
Area num: 5, field_xy: [570, 697, 0, 127]
Area num: 6, field_xy: [684, 811, 0, 127]
Area num: 7, field_xy: [798, 925, 0, 127]
Area num: 8, field_xy: [912, 1039, 0, 127]
Area num: 9, field_xy: [1026, 1153, 0, 127]
Area num: 10, field_xy: [1140, 1267, 0, 127]
Area num: 11, field_xy: [1174, 1301, 0, 127]
Area num: 12, field_xy: [0, 127, 114, 241]
Area num: 13, field_xy: [114, 241, 114, 241]
Area num: 14, field_xy: [228, 355, 114, 241]
Area num: 15, field_xy: [342, 469, 114, 241]
Area num: 16, field_xy: [456, 583, 114, 241]
Area num: 17, field_xy: [570, 697, 114, 241]
Area num: 18, field_xy: [684, 811, 114, 241]
Area num: 19, field_xy: [798, 925, 114, 241]
Area num: 20, field_xy: [912, 1039, 114, 241]
Area num: 21, field_xy: [1026, 1153, 114, 24

#### Read 30 random segmentation masks

In [7]:
filenames = model.data_generator.cell_mask_filenames

In [8]:
random_filenames = random.choices(filenames, k=1)

In [9]:
cell_masks = []
for i, file in enumerate(random_filenames,0):
    img = imread(file)
    #img = rotate(img, random_angles[i], reshape=False)
    cell_masks.append(img)

In [10]:
img.shape

(1041, 1302)

In [11]:
cell_masks_batch = np.stack(cell_masks)

In [12]:
cell_masks_batch.shape

(1, 1041, 1302)

In [13]:
plt.figure()
plt.imshow(cell_masks_batch[0])
plt.show()

In [14]:
prob_map = (cell_masks_batch > 0) * 0.003125

In [15]:
plt.figure()
plt.imshow(prob_map[0])
plt.show()

In [16]:
data_gen = DataSimulator(psf_params, simulation_params, hardware_params)

In [17]:
S, X_os, Y_os, Z, I, field_xy = data_gen.sampling(batch_size=1, prob_map=gpu(prob_map), local_context=None,
                                                  iter_num=None)

In [18]:
S.shape

torch.Size([1, 1, 1041, 1302])

In [19]:
plt.figure()
plt.imshow(S[0][0].cpu().numpy())
plt.show()

In [40]:
index = 0
fig, ax = plt.subplots(nrows=1, ncols=6)
ax[0].imshow(S[index][0].cpu().numpy())
ax[0].set_title('Dot location')
x_os_img = ax[1].imshow(X_os[index][0].cpu().numpy())
ax[1].set_title('X offset')
#fig.colorbar(x_os_img, ax=ax[1])
y_os_img = ax[2].imshow(Y_os[index][0].cpu().numpy())
ax[2].set_title('Y offset')
#fig.colorbar(y_os_img, ax=ax[2])
z_vals_img = ax[3].imshow(Z[index][0].cpu().numpy())
ax[3].set_title('Z value')
#fig.colorbar(z_vals_img, ax=ax[3])
i_vals_img = ax[4].imshow(I[index][0].cpu().numpy())
ax[4].set_title('Intensity Value')
#fig.colorbar(i_vals_img, ax=ax[4])
ax[5].imshow(cell_masks_batch[index])
ax[5].set_title('cell mask')
plt.show()

In [20]:
imgs_sim, single_psfs = data_gen.simulate_psfs(S, X_os, Y_os, Z, I)

In [21]:
imgs_sim.shape, single_psfs.shape

(torch.Size([1, 1, 1041, 1302]), torch.Size([320, 40, 40]))

In [24]:
index = 14
fig, ax = plt.subplots(nrows=1, ncols=5)
ax[0].imshow(prob_map[index])
ax[0].set_title('Probabilty map')
ax[1].imshow(cell_masks_batch[index])
ax[1].set_title('Cell mask sampled')
ax[2].imshow(imgs_sim[index, 0].cpu().numpy())
ax[2].set_title('Simulated PSFs')
ax[3].imshow(S[index, 0].cpu().numpy())
ax[3].set_title('Emitter pixel locs')
ax[4].imshow(I[index, 0].cpu().numpy() * psf_params['photon_scale'])
ax[4].set_title('Intensity values')
plt.show()

In [22]:
S.shape

torch.Size([1, 1, 1041, 1302])

In [23]:
xyzi = torch.cat([X_os[:, :, None], Y_os[:, :, None], Z[:, :, None], I[:, :, None]], 2)
xyzi = xyzi[:, 0]

In [24]:
S = S[:, 0]

In [25]:
s_inds = tuple(S.nonzero().transpose(1, 0))
# get these molecules' sub-pixel xy offsets, z positions and photons
xyzi_true = xyzi[s_inds[0], :, s_inds[1], s_inds[2]]

In [26]:
xyzi_true

tensor([[ 0.2830, -0.3434,  0.0508,  0.3924],
        [-0.4633, -0.4936, -0.5779,  0.7539],
        [-0.3625, -0.1138, -0.8307,  0.6108],
        ...,
        [-0.1428, -0.2149,  0.0137,  0.8335],
        [-0.4844,  0.1045, -0.7952,  0.7211],
        [ 0.0885,  0.1410,  0.1395,  0.6714]], device='cuda:0')

In [27]:
xyzi_true[:, 0] += s_inds[2] + 0.5

In [28]:
xyzi_true[:, 1] += s_inds[1] + 0.5

In [29]:
pos_tar = xyzi_true[:, :3]

In [30]:
pos_tar.shape

torch.Size([320, 3])

In [31]:
pos_tar[:, 2] *= 500

In [32]:
pos_tar

tensor([[ 8.8978e+02,  9.9157e+01,  2.5401e+01],
        [ 6.9104e+02,  1.0001e+02, -2.8897e+02],
        [ 6.7814e+02,  1.0139e+02, -4.1536e+02],
        [ 8.9008e+02,  1.0129e+02,  2.1743e+02],
        [ 9.8598e+02,  1.0132e+02,  2.4131e+02],
        [ 4.7616e+02,  1.0216e+02, -2.1110e+02],
        [ 5.4572e+02,  1.0227e+02, -3.7735e+02],
        [ 8.3047e+02,  1.0226e+02, -3.4997e+02],
        [ 5.1662e+02,  1.0320e+02,  4.5673e+01],
        [ 6.7794e+02,  1.0307e+02,  2.8634e+02],
        [ 9.8056e+02,  1.0372e+02, -2.0642e+01],
        [ 8.0931e+02,  1.0498e+02,  7.0203e+01],
        [ 9.1322e+02,  1.0495e+02, -2.7816e+02],
        [ 8.2832e+02,  1.0582e+02, -4.0355e+02],
        [ 1.0915e+03,  1.0581e+02, -3.0961e+02],
        [ 4.6408e+02,  1.0616e+02, -4.0031e+01],
        [ 7.7638e+02,  1.0692e+02,  3.5670e+02],
        [ 1.0731e+03,  1.0606e+02, -1.8458e+02],
        [ 4.3773e+02,  1.0765e+02, -1.5468e+02],
        [ 7.0723e+02,  1.0873e+02, -7.7269e+01],
        [ 8.0213e+02

In [33]:
xyzi_true[0, 3]

tensor(0.3924, device='cuda:0')

In [34]:
plt.figure()
plt.imshow((single_psfs[0] * 0.3924 *  3000).numpy())
plt.show()

In [35]:
from cellbgnet.utils.plot_funcs import PlotFrameCoord

In [36]:
imgs_sim.shape

torch.Size([1, 1, 1041, 1302])

In [37]:
plt.figure()
PlotFrameCoord(imgs_sim[0].cpu(), pos_tar=pos_tar.cpu(),plot_colorbar_frame=True, annotate_tar_z=True).plot()
plt.show()

In [25]:
plt.figure()
plt.imshow(imgs_sim[13, 0].cpu().numpy())
plt.show()

### Noise adding inside and outside cells

In [52]:

dists = np.zeros_like(cell_masks_batch)
for i in range(len(cell_masks_batch)):
    dists[i] = edt.edt(cell_masks_batch[i])

In [53]:
dists.shape

(30, 1041, 1302)

In [54]:
index = 12
fig, ax = plt.subplots(nrows=1, ncols=3)
ax[0].imshow(cell_masks_batch[index])
ax[0].set_title('Cell mask')
ax[1].imshow(dists[index])
ax[1].set_title('Edt')
ax[2].imshow(prob_map[index])
ax[2].set_title('Prob map')
plt.show()

In [55]:
def mean_bg_mask(dists, mean_map):
    dists_copy = np.copy(dists)
    for edt_val, mean_bg in mean_map.items():
        dists_copy[dists == int(edt_val)] = mean_bg
    return dists_copy

In [56]:
def variance_bg_mask(dists, variance_map):
    dists_copy = np.copy(dists)
    for edt_val, stddev_bg in variance_map.items():
        dists_copy[dists == int(edt_val)] = stddev_bg
    return dists_copy

In [57]:
dists = np.ceil(dists)

In [58]:
#mean_bg_cells = mean_bg_mask(dists, mean_map)
#stddev_bg_cells = variance_bg_mask(dists, stddev_map)
fitted_beta_map = edt_noise_map['betas']
fitted_alpha_map = edt_noise_map['alphas']
alpha_bg_cells = mean_bg_mask(dists, fitted_alpha_map)
beta_bg_cells = variance_bg_mask(dists, fitted_beta_map)

In [59]:
index = 10
fig, ax = plt.subplots(nrows=1, ncols=2)
#ax[0].imshow(mean_bg_cells[index])
ax[0].imshow(alpha_bg_cells[index])
ax[0].set_title('Alpha values')
#ax[0].set_title('Mean bg values')
#ax[1].imshow(stddev_bg_cells[index])
ax[1].imshow(beta_bg_cells[index])
#ax[1].set_title('Std dev bg values')
ax[1].set_title('Beta values')
plt.show()

In [60]:
alpha_t = torch.from_numpy(alpha_bg_cells)
beta_t = 1.0/torch.from_numpy(beta_bg_cells)

In [61]:
alpha_t.shape, beta_t.shape

(torch.Size([30, 1041, 1302]), torch.Size([30, 1041, 1302]))

In [62]:
m = torch.distributions.gamma.Gamma(concentration=alpha_t, rate=beta_t)
sample = m.sample()

In [63]:
index = 10
fig, ax = plt.subplots(nrows=1, ncols=3)
#ax[0].imshow(mean_bg_cells[index])
ax[0].imshow(alpha_bg_cells[index])
ax[0].set_title('Alpha bg cells')
#ax[0].set_title('Mean bg values')
#ax[1].imshow(stddev_bg_cells[index])
#ax[1].set_title('Std dev bg values')
ax[1].imshow(beta_bg_cells[index])
ax[1].set_title('Beta bg cells')
img = ax[2].imshow(sample[index].cpu().numpy(), cmap='gray')
ax[2].set_title('Sampled bg values')
fig.colorbar(img, ax=ax[2])
plt.show()

In [64]:
index = 15
plt.figure()
plt.imshow(sample[index].cpu().numpy(), cmap='gray')
plt.show()

### Dot overlay

In [65]:
baseline = 103.0
e_per_adu = 0.39
qe = 0.95
bg_photons = (sample - baseline) * e_per_adu / qe

In [66]:
bg_photons = bg_photons[:, None]

In [67]:
bg_photons = torch.clamp(bg_photons, min=0.0)

In [68]:
imgs_sim.shape, bg_photons.shape

(torch.Size([30, 1, 1041, 1302]), torch.Size([30, 1, 1041, 1302]))

In [69]:
bg_photons = bg_photons.to('cuda:0')

In [70]:
plt.figure()
plt.imshow(bg_photons[index, 0].cpu().numpy())
plt.show()

In [71]:
after_dots = imgs_sim + bg_photons

In [72]:
index = 10
fig, ax = plt.subplots(nrows=1, ncols=6)
ax[0].imshow(prob_map[index])
ax[0].set_title('Probabilty map')
ax[1].imshow(cell_masks_batch[index])
ax[1].set_title('Cell mask sampled')
ax[2].imshow(imgs_sim[index, 0].cpu().numpy())
ax[2].set_title('Simulated PSFs')
ax[3].imshow(S[index, 0].cpu().numpy())
ax[3].set_title('Emitter pixel locs')
ax[4].imshow(I[index, 0].cpu().numpy() * psf_params['photon_scale'])
ax[4].set_title('Intensity values')
ax[5].imshow(after_dots[index][0].cpu().numpy(), cmap='gray')
ax[5].set_title('After overlaying dots')
plt.show()

### Adding camera noise

In [73]:
RN = 2.2

In [74]:
after_dots = torch.distributions.Poisson(after_dots * qe).sample()

In [75]:
zeros = torch.zeros_like(after_dots)

In [76]:
readout_noise = torch.distributions.Normal(zeros, zeros + RN).sample()

In [77]:
after_dots = after_dots + readout_noise

In [78]:
final_dots_for_net = torch.clamp((after_dots/ e_per_adu) + baseline, min=0)

In [79]:
index = 10
fig, ax = plt.subplots(nrows=1, ncols=7)
ax[0].imshow(sample[index].cpu().numpy(), cmap='gray')
ax[0].set_title('Sampled background ADU')
ax[1].imshow(cell_masks_batch[index])
ax[1].set_title('Cell mask sampled')
ax[2].imshow(imgs_sim[index, 0].cpu().numpy())
ax[2].set_title('Simulated PSFs')
ax[3].imshow(S[index, 0].cpu().numpy())
ax[3].set_title('Emitter pixel locs')
ax[4].imshow(I[index, 0].cpu().numpy() * psf_params['photon_scale'])
ax[4].set_title('Intensity values')
ax[5].imshow(after_dots[index][0].cpu().numpy(), cmap='gray')
ax[5].set_title('After overlaying dots (photons)')
ax[6].imshow(final_dots_for_net[index][0].cpu().numpy(), cmap='gray')
ax[6].set_title('After camer noise (ADU)')
plt.tight_layout()
plt.show()

In [80]:
plt.figure()
plt.imshow(final_dots_for_net[index][0].cpu().numpy(), cmap='gray')
plt.show()

In [ ]:
plt.figure()
plt.imshow()

In [54]:
index = 3
plt.figure()
plt.imshow(final_dots_for_net[index][0].cpu().numpy(), cmap='gray')
plt.colorbar()
plt.show()

In [51]:
final_dots_for_net[index][0].cpu().numpy().max()

331.9748

In [52]:
final_dots_for_net[index][0].cpu().numpy().min()

76.83658